#  Jogo BreakThrough

## Introdução à Inteligência Artificial edição 2022/23
### Projeto para avaliação

<img src=".\images\Picture0.png" alt="Drawing" style="width: 200px;"/>

### Grupo: 40

#### Elementos do Grupo

Nome: Francisco Correia

Número: 54685

Nome: Francisco Maia

Número: 55855

Nome: Alexandre Fonseca

Número: 55955

## Introdução
Este é um esqueleto do relatório que podem naturalmente expandir, colocando mais células de texto e/ou de código.

## Formulação do Jogo BreakThrough em termos de estados e de operadores

### Descrição da representação dos estados do jogo

## TODO

In [ ]:
from time import time
from jogos import *
from jogar import *

class EstadoBT_40:
    def __init__(self, to_move, utility, board, whites, blacks, moves=None):
        self.to_move = to_move
        self.utility = utility
        self.board = board
        self.whites = whites
        self.blacks = blacks
        self.moves = moves

    def __str__(self):
        player_chars = [".", "W", "B"]  # 0, WHITES and BLACKS
        board_str = ["-----------------"]
        for i in range(len(self.board), 0, -1):
            board_str.append(
                f"{i}|" + " ".join(map(lambda x: player_chars[x], self.board[i - 1]))
            )
        board_str.append("-+---------------")
        board_str.append(" |a b c d e f g h")
        return "\n".join(board_str)


class JogoBT_40(Game):
    WHITE = 1
    BLACK = 2

    def __init__(self, n=8):
        self.n = n
        whites = set((row, col) for row in range(2) for col in range(n))
        blacks = set((row, col) for row in range(n - 2, n) for col in range(n))
        board = [[0 for _ in range(n)] for _ in range(n)]
        for x, y in whites:
            board[x][y] = JogoBT_40.WHITE
        for x, y in blacks:
            board[x][y] = JogoBT_40.BLACK
        to_move = JogoBT_40.WHITE
        self.action_dict = self.compute_action_dict(n)
        self.initial = EstadoBT_40(to_move, 0, board, whites, blacks)

    def actions(self, state: EstadoBT_40):
        if state.moves:
            return state.moves
        moves = set()
        if state.to_move == JogoBT_40.WHITE:
            pieces, pieces_opponent = state.whites, state.blacks
        else:
            pieces, pieces_opponent = state.blacks, state.whites
        for pos in pieces:
            for target, move in self.action_dict[pos][state.to_move].items():
                # can't eat opponent's piece if it is directly in front of ours
                if target[1] == pos[1] and target in pieces_opponent:
                    continue
                if target not in pieces:  # don't eat our own pieces
                    moves.add(move)
        state.moves = sorted(moves)
        return state.moves

    def display(self, state):
        print(state)
        if not self.terminal_test(state):
            print(f'--NEXT PLAYER: {"W" if state.to_move == JogoBT_40.WHITE else "B"}')

    def executa(self, state: EstadoBT_40, valid_actions: "list[str]"):
        """Executa várias jogadas sobre um estado dado.
        Devolve o estado final."""
        result = state
        for move in valid_actions:
            result = self.result(result, move)
        return result

    def result(self, state: EstadoBT_40, move):
        board = [row[:] for row in state.board]  # deepcopy
        (old_row, old_col), (new_row, new_col) = self.convert_move(move)
        board[new_row][new_col] = state.to_move
        board[old_row][old_col] = 0
        whites, blacks = state.whites.copy(), state.blacks.copy()

        if state.to_move == JogoBT_40.WHITE:
            whites.remove((old_row, old_col))
            whites.add((new_row, new_col))
            blacks.discard((new_row, new_col))
            to_move = JogoBT_40.BLACK
        else:
            blacks.remove((old_row, old_col))
            blacks.add((new_row, new_col))
            whites.discard((new_row, new_col))
            to_move = JogoBT_40.WHITE

        return EstadoBT_40(
            to_move,
            self.compute_utility(new_row, state.to_move),
            board,
            whites,
            blacks,
        )

    def terminal_test(self, state: EstadoBT_40):
        # Os próprios jogos em jogar.py lidam com
        # os casos em que já não há jogadas (peças).
        return state.utility != 0

    def utility(self, state: EstadoBT_40, player):
        # W: 1, B: -1
        return state.utility if player == JogoBT_40.WHITE else -state.utility

    def compute_utility(self, row, player):
        """
        Devolve a utilidade para um dado estado:
        (1: vitória, -1: derrota, 0: não terminou)

        Pressupõe-se que a função é chamada unicamente quando
        é criado um novo estado e que o valor de `row` resulta
        de uma jogada válida para o `player` em questão.
        """
        if 0 < row < self.n - 1:
            return 0
        return 1 if player == JogoBT_40.WHITE else -1

    def compute_action_dict(self, n):
        """Devolve um dicionário que associa uma posição `(x, y)`
        usada no estado interno às ações disponíveis nessa posição.

        As associações são feitas em função do jogador
        (`1` para peças brancas, `2` para peças pretas).

        As ações são devolvidas num dicionário em que as chaves são
        a posição do destino e os valores são a ação em si.

        Exemplo de ações para uma peça em `(1,1)` (visualmente, `b2`):
        ```python
        jogo = JogoBT_40()
        acoes_em_b2 = jogo.action_dict[(1, 1)]
        jogo.action_dict[(1, 1)] == (
            None,  # padding para indexar com state.to_move sem offset
            {(2, 0): "b2-a3", (2, 1): "b2-b3", (2, 2): "b2-c3"}, # jog. 1
            {(0, 0): "b2-a1", (0, 1): "b2-b1", (0, 2): "b2-c1"}, # jog. 2
        )
        ```"""

        def in_board(x):
            return 0 <= x < n

        ret = {}
        a_ord = ord("a")
        for row in range(n):
            for col in range(n):
                black_moves = {}
                white_moves = {}
                for i in filter(in_board, (row - 1, row + 1)):
                    # white pieces move upwards, black pieces downwards
                    moves = white_moves if i > row else black_moves
                    for j in filter(in_board, (col - 1, col, col + 1)):
                        moves[(i, j)] = "-".join(
                            [f"{chr(col + a_ord)}{row + 1}", f"{chr(j + a_ord)}{i + 1}"]
                        )
                # use None to pad tuple so we can use to_move as index
                ret[(row, col)] = (None, white_moves, black_moves)
        return ret

    # ? Usar dict inverso para mais desempenho
    def convert_move(self, move):
        """Converte uma ação no formato do enunciado
        para uma ação representada no estado interno.
        Exemplo: "a1-b2" -> ((0, 0), (1, 1))"""
        ord_a = ord("a")
        (old_col, old_row), (new_col, new_row) = move.split("-")
        return (
            (int(old_row) - 1, ord(old_col) - ord_a),
            (int(new_row) - 1, ord(new_col) - ord_a),)

### Testes da formulação

#### Situações iniciais dos jogos
Uso do construtor e "display" de jogos iniciais

Construção de um novo jogo com a situação inicial seguinte:

<img src=".\images\Picture1.png" alt="Drawing" style="width: 150px;"/>
<p style="text-align: center;">Figura 1</p>

### Método Display()
Tendo em conta o que foi explicado acima, a construção de um novo jogo tem como objetivo definir quais as posições iniciais das peças brancas e pretas, as brancas vão preencher as primeiras duas linhas e as pretas as últimas duas, o primeiro jogador a jogar, que será sempre o que joga com as peças brancas, e criar o dicionário de jogadas, que vai ser util para traduzir um par de cordenadas em jogadas do estilo "a1-a2".

A função compute_action_dict() devolve um dicionário que associa uma posição `(x, y)` usada no estado interno às ações disponíveis nessa posição.

As associações são feitas em função do jogador (`1` para peças brancas, `2` para peças pretas).

As ações são devolvidas num dicionário em que as chaves são a posição do destino e os valores são a ação em si.

In [ ]:
j1 = JogoBT_40()

Eis o display desse estado inicial do jogo:

In [ ]:
j1.display(j1.initial)

### Método actions()

Podemos verificar quais as ações possiveís para o próximo jogador, neste caso 'W', a partir do método actions. Este método foi definido para iterar sobre todas as peças do jogador a jogar e de seguida iterar pelo dicionário de ações.
O dicionário está definido da seguinte forma:

```Python
(1,1) -> (
    None,
    1 (Whites): {(2,0): b2-a3, (2,1): b2-b3,(2,2): b2-c3},
    2 (Blacks): {(0,0): b2-a1, (0,1): b2-b1,(0,2): b2-c1}
)
```
Isto é, para a posição (1,1) uma peça branca pode mover-se para (2,0), (2,1), (2,2) e uma peça preta para (0,0), (0,1), (0,2).

Durante a iteração pelas jogadas que podem ser executadas verificamos:
- Se na posição "à frente" da posição inicial da peça existe uma peça do adversário
- Se não existem peças do jogador na posição para as posições que a peça se pode mover

A jogada só é válida se **ambas** as condições forem falsas.

In [ ]:
to_move = j1.initial.to_move
player = 'W' if to_move == 1 else 'B'
print("Next Player:", player)
print("Ações possíveis:", j1.actions(j1.initial))

## Jogos entre jogadores simples
Nesta secção irão realizar alguns jogos, para verificar a modelização

Jogo entre dois jogadores random:

In [ ]:
j2 = JogoBT_40()
player1 = Jogador("Randall", random_player)
player2 = Jogador("Randy", random_player)
game_to_display = joga11(j2, player1, player2)
mostraJogo(j2, game_to_display, False)

Jogo entre um jogador random e um jogador alfabeta, com depth 2, que utiliza como eval_fun a função utility:

In [ ]:
player3 = JogadorAlfaBeta("Alfaiate", 2, j2.utility)
mostraJogo(j2, joga11(j2, player3, player1))

Campeonato entre 4 jogadores, dois random's e dois alfabeta, com depth 2, que utilizam a função utility como eval_fun:

In [ ]:
player4 = JogadorAlfaBeta("Alberta", 2, j2.utility)
faz_campeonato(j2, [player1, player2, player3, player4], 10)

Faça o display de um dos jogos realizados atrás

In [ ]:
jogadores, jogadas, vencedor = game_to_display
print("Jogo entre", jogadores[0], "e", jogadores[1], "\n")
print("As jogadas efetuadas foram: ", jogadas, "\n")
print("Vencedor do jogo:", jogadores[vencedor - 1])
estado_to_display = j2.initial
for move in jogadas:
    j2.display(estado_to_display)
    estado_to_display = j2.result(estado_to_display, move)

## Exemplos de jogadores alfabeta
 Descreva e teste nesta secção as várias funções de avaliação desenvolvidas tanto para o ataque como para a defesa.

### Belarmino

Primeiramente, antes de definirmos as nossas próprias funções de avaliação, definimos o belarmino:

In [ ]:
def f_aval_belarmino(estado: EstadoBT_40, jogador):
    res = 0
    n = len(estado.board)
    if jogador == JogoBT_40.WHITE:
        for row, _ in estado.whites:
            x = row + 1
            res += x**x
    else:
        for row, _ in estado.blacks:
            x = n - row
            res += x**x
    return res

### Funções de avaliação desenvolvidas

O próximo jogador desenvolvido, vamos chamar-lhe **Marco**, este jogador vai avaliar:
1. **Vitória**: Se o jogador ja venceu
2. **Vitória iminente**: Se é possível ganhar em uma jogada, ou seja, se temos uma peça na penúltima linha, e esta peça não pode ser "comida" - ```função threat() ```
3. **HomeGround**: Se a peça está na primeira casa, para peças brancas na linha 1, para peças pretas na linha 8
4. **Colunas vazias**: Se existem colunas vazias, isto é, que não contenham nenhuma peça do jogador
5. **Peças do oponente**: O número de peças do oponente

- Decidimos valorizar bastante o **ponto 1** e o **ponto 2** pois ambos são casos em que chegamos facilmente e rapidamente à vitória.
- O **ponto 3**, é uma forma de nos defendermos frente a um adversário muito atacante, como o **belarmino**.
- O **ponto 4**, tem como objetivo manter as em colunas diferentes, para não acontecer o caso de ficar com demasiadas peças numa só coluna
- O **ponto 5**, após alguns testes verificamos ser um dos que mais influencia, pois quanto menos peças o adversário tem mais dificil é este vencer, dai desvalorizarmos muito para quantas mais peças o oponente tiver


In [ ]:
def f_aval_jogador_Marco(estado: EstadoBT_40, jogador):
    res = 0
    n = len(estado.board)
    home, target, k = (0, n - 1, -1) if jogador == JogoBT_40.WHITE else (n - 1, 0, 1)

    if jogador == JogoBT_40.WHITE:
        pieces, pieces_opponent = estado.whites, estado.blacks
    else:
        pieces, pieces_opponent = estado.blacks, estado.whites

    # Verificar se existem colunas vazias
    occupied_cols = set()

    # Player win
    if target in map(lambda x: x[0], list(pieces)):
        return 500000

    for row, col in pieces:
        occupied_cols.add(col)
        # res += piece_value(estado, row, col)
        # One move to win
        if row == target + k and not threat(estado, row, col):
            res += 10000
        # Homeground piece
        elif row == home:
            res += 150

    res -= (n - len(occupied_cols)) * 20
    res -= len(pieces_opponent) * 20
    return res

def threat(estado: EstadoBT_40, row, col):
    if estado.to_move == JogoBT_40.WHITE:
        pieces_opponent = estado.blacks
        row += 1
    else:
        pieces_opponent = estado.whites
        row -= 1
    return (row, col - 1) in pieces_opponent or (row, col + 1) in pieces_opponent

A última iteração do nosso jogador é o **Heurácio**. Baseia-se no jogador anterior mas incorpora uma nova função na função anterior: `piece_value().`

Esta função valoriza os seguintes critérios:
- `(+)` **ligações horizontais**: existência de peças amigáveis adjacentes à peça atual na horizontal
- `(+)` **ligações verticais**: existência de peças amigáveis adjacentes à peça atual na vertical
- `(+)` **proteção**: existência de peças amigáveis que podem contra-atacar caso a nossa peça seja comida pelo adversário
- `(-)` **vulnerabilidade**: possibilidade da nossa peça ser comida pelo adversário
- `(+)` **distância até à vitória**: a peça estar na penúltima ou na antepenúltima fila, desde que **não** se encontre **vulnerável**
- `(+)` **valor intrínseco**: atribuem-se pontos a todas as peças (isto é, quantas mais, melhor)

In [ ]:
def piece_value(estado: EstadoBT_40, row, col):
    res = 0
    n = len(estado.board)

    piece_in_danger = False
    if estado.to_move == JogoBT_40.WHITE:
        pieces = estado.whites
        pieces_opponent = estado.blacks
        friend_row, opp_row = row - 1, row + 1
        second_row, third_row = n - 2, n - 3  # from winning row
        row_value = row + 1

    else:
        pieces = estado.blacks
        pieces_opponent = estado.whites
        friend_row, opp_row = row + 1, row - 1
        second_row, third_row = 1, 2  # from winning row
        row_value = n - row

    # * Verify horizontal connections
    if (row, col - 1) in pieces or (row, col + 1) in pieces:
        res += 15

    # * Verify vertical connections
    if (row - 1, col) in pieces or (row + 1, col) in pieces:
        res += 5

    # * Verify if piece is protected (can counter-attack)
    if (friend_row, col - 1) in pieces or (friend_row, col + 1) in pieces:
        res += 15

    # * Verify if piece can be attacked
    if (opp_row, col - 1) in pieces_opponent or (opp_row, col + 1) in pieces_opponent:
        # Peças mais avançadas e que não podem ser atacadas valem mais
        res -= 15
        piece_in_danger = True

    if not piece_in_danger:
        if row == third_row:
            res += 5
        elif row == second_row:
            res += 20

    res += row_value * 10

    return res


Usamos a função `piece_value()` no novo jogador (linha 20):

In [ ]:
def f_aval_jogador_Heuracio(estado: EstadoBT_40, jogador):
    res = 0
    n = len(estado.board)
    home, target, k = (0, n - 1, -1) if jogador == JogoBT_40.WHITE else (n - 1, 0, 1)

    if jogador == JogoBT_40.WHITE:
        pieces, pieces_opponent = estado.whites, estado.blacks
    else:
        pieces, pieces_opponent = estado.blacks, estado.whites

    # Verificar se existem colunas vazias
    occupied_cols = set()

    # Player win
    if target in map(lambda x: x[0], list(pieces)):
        return float("+inf")

    for row, col in pieces:
        occupied_cols.add(col)
        res += piece_value(estado, row, col)
        # One move to win
        if row == target + k and not threat(estado, row, col):
            res += 10000
        # Homeground piece
        elif row == home:
            res += 150

    res -= (n - len(occupied_cols)) * 20
    res -= len(pieces_opponent) * 80
    return res

## Exemplos de jogos entre alguns desses jogadores e o Belarmino

### Realização de campeonato com depth 1

Dentro deste campeonato vão existir jogos entre os nosso jogadores também, mas o que pretendemos demonstrar é os resultados dos jogadores frente ao belarmino

In [ ]:
depth = 1
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [belarmino, marco, heuracio], 10)

### Realização de campeonato com depth 2
Dentro deste campeonato vão existir jogos entre os nosso jogadores, mas o que pretendemos demonstrar é os resultados dos jogadores frente ao belarmino

In [ ]:
depth = 2
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [belarmino, marco, heuracio], 10)

### Realização de jogos com depth 3

*Nota: Não é realizado um campeonato pois este é demorado*

In [ ]:
depth = 3
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)

mostraJogo(jogo, joga11com_timeout(jogo, belarmino, marco, 10), False)
mostraJogo(jogo, joga11com_timeout(jogo, heuracio, belarmino, 10), False)

## Exemplos de jogos entre dois dos vários jogadores desenvolvidos

### Campeonato com depth 1

In [ ]:
depth = 1
jogo = JogoBT_40()
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [marco, heuracio], 10)

### Campeonato com depth 2

In [ ]:
depth = 2
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [marco, heuracio], 10)

### Campeonato com depth 3

In [ ]:
depth = 3
jogo = JogoBT_40()
belarmino = JogadorAlfaBeta("Belarmino", depth, f_aval_belarmino)
marco = JogadorAlfaBeta("Marco", depth, f_aval_jogador_Marco)
heuracio = JogadorAlfaBeta("Heuracio", depth, f_aval_jogador_Heuracio)
faz_campeonato(jogo, [marco, heuracio], 10)

*Nota: Ao testar, verificamos que é possível realizar jogos com depth 4, mas não vamos incluir pois são muito demorados*

## Processo de selecção dos jogadores para o torneio
Descreva o processo de selecção dos jogadores campeões, para entrar no campeonato "todos contra todos".



*...aqui* 

<span style="color:magenta"> É a parte mais importante do relatório, justificando porque convocaram o vosso Ronaldo para o campeonato. Se jogaram com vários jogadores (ou seja, várias funções de avaliação) e fizeram um torneio privado de selecção, podem apresentar aqui uma tabela com esses dados. </span>